In [186]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import SGDRegressor
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import VarianceThreshold

# Assignment 2 - Regression
### Predict the TOTAL COMPENSATION for this year. 

The data file provided is a salary survey for tech workers in Europe. We want to predict the total amount of compensation they bring in each year, based off of the details of their work. 

Some notes that will be important:
<ul>
    <li>The total compensation will need to be constructed, there is a column for salary, "Yearly brutto salary (without bonus and stocks) in EUR", as well as a column for bonus compensation, "Yearly bonus + stocks in EUR". 
    <li>Some categorical variables will need some work, and there isn't generally an exact answer. The main concern is things with categories that have a bunch of values with a very small count. For example, if there is only 1 person in City X, then that value likely needs to be addressed. We don't want it encoded into a new column of one 1 and thousands of 0s. 
    <li>There is an article exploring some of the data here: https://www.asdcode.de/2021/01/it-salary-survey-december-2020.html
    <li>Imputation and a bit of data manipulation will be required. 
    <li>Use any regression method you'd like. Some ones are closely related to what we've done, you may want to look at them, e.g. ExtraTreesRegressor. 
    <li>Initial accurracy, and potentially final accuracy, may not be great. When I made a plain model will little optimization the errors were large and the R2 was low. There is lots of room for optimization. 
    <li>Research challenge - try some work on the target, look into TransformedTargetRegressor and see if that helps. Recall in stats when we had skewed distributions... Maybe it helps, maybe it doesn't. 
    <li>EDA and data prep are up to you - you'll probably need to do a little exploring to figure out what cleanup is needed. When I did it, I did things kind of iteratively when I did it. For example, look at the value counts, figure out how to treat the different categories, clean something up, look at the results, potentially repeat if needed. After you figure out what needs to be done, you may be able to take some of those steps and incorporate them into a pipeline to be cleaner....
    <li><b>CRITICAL - Please make sure your code runs with RUN ALL. It should load the data that you're given, do all the processing, and spit out results. Comment out or remove anything that you've cleaned up and don't need - e.g. if you scaled a value manually, then moved that into a pipeline, don't leave the original scaling code active when the file is run.</b>
</ul>

### Details and Deliverables

You'll need to build code to produce the predictions. In particular, there's a few things that'll be marked:
<ul>
    <li>Please add a markdown cell at the bottom, and put in a few notes addressing the following:
    <ul>
        <li> Accuracy of your models with/without feature selection. Include both train/test for each. Please use R2 and RMSE. 
        <li> Feature Selection - Please identify what you did for feature selection. No need for a long explaination, something along the lines of "I did X, and the result was that 4 features were removed". Try at least 2 things. 
        <li> Hyperparameter Changes / Grid Search Improvements. What did you try, and why. Similar explaination to above, short. 
        <li> Overall this section should be roughly as long as this intro block - just outline what the results were, what you did to improve, and the results after. 
        <li> If you could use titles/bullet points I'd really appreciate it. 
    </ul>
    <li>Grade Breakdown:
    <ul>
        <li> Code is readable, there are comments: 20%
        <li> Explaination as defined above: 60% (20% each point)
        <li> Accuracy: 20% As compared to everyone else. This will be generously graded, I won't be surprised if overall accuracy is low for most people. 
    </ul>
</ul>

In [187]:
#Load Data
df = pd.read_csv("data/Euro_Salary.csv")
df.describe(include="all").T

,count,unique,top,freq,mean,std,min,25%,50%,75%,max
Timestamp,1253,1248,24/11/2020 13:55:19,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Age,1226.0,NaN,NaN,NaN,32.509788,5.663804,20.0,29.0,32.0,35.0,69.0
Gender,1243,3,Male,1049,NaN,NaN,NaN,NaN,NaN,NaN,NaN
City,1253,119,Berlin,681,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Position,1247,148,Software Engineer,387,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Total years of experience,1237,48,10,138,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Years of experience in Germany,1221,53,2,195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Seniority level,1241,24,Senior,565,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Your main technology / programming language,1126,256,Java,184,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Other technologies/programming languages you use often,1096,562,Javascript / Typescript,44,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [188]:
df.sample(10)

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
20,24/11/2020 11:21:10,34.0,Male,Berlin,DevOps,13,5,Senior,NaN,NaN,103000.0,NaN,30,Full-time employee,Unlimited contract,English,1000+,Product
299,24/11/2020 15:42:31,28.0,Male,Berlin,Backend Developer,7,1,Senior,PHP,"Javascript / Typescript, .NET, SQL, Azure, Kub...",70000.0,NaN,30,Full-time employee,Unlimited contract,English,101-1000,Product
743,26/11/2020 09:20:23,29.0,Male,Berlin,Frontend Developer,3,0,Middle,JavaScript,PHP,48000.0,0,26,Full-time employee,Unlimited contract,English,51-100,Consulting / Agency
664,25/11/2020 18:24:15,30.0,Male,Berlin,Software Engineer,8,2,Senior,JS,NaN,60000.0,0,~25,Full-time employee,Unlimited contract,English,51-100,Product
323,24/11/2020 17:21:09,32.0,Male,Munich,Backend Developer,11,3.5,Senior,PHP,"PHP, Javascript / Typescript, Java / Scala, Do...",81200.0,81200,30,Full-time employee,Unlimited contract,English,1000+,Product
423,24/11/2020 20:29:50,39.0,Male,Berlin,DevOps,15,3,Senior,Python,"Python, Ruby, Java / Scala, Go, Rust, AWS, Kub...",70000.0,0,21,Full-time employee,Unlimited contract,English,101-1000,Product
930,30/11/2020 11:16:18,30.0,Male,Cracow,Business Analyst,7,0,Senior,SQL,"Python, AWS, Google Cloud, Azure, Alteryx, SAS",68500.0,70000,26,Full-time employee,Unlimited contract,English,51-100,Consulting / Agency
279,24/11/2020 15:05:21,39.0,Male,Berlin,Product Manager,10,5,Lead,NaN,NaN,120000.0,200000,25,Full-time employee,Unlimited contract,English,1000+,Product
14,24/11/2020 11:18:40,28.0,Male,Berlin,Designer (UI/UX),11,2,Senior,NaN,NaN,75000.0,9400,30,Full-time employee,Unlimited contract,English,1000+,Product
468,25/11/2020 00:01:59,31.0,Female,Berlin,Product Manager,NaN,1,Senior,NaN,NaN,60000.0,NaN,20,Full-time employee,Unlimited contract,English,1000+,Product


In [189]:
#df.info()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1253 entries, 0 to 1252
Data columns (total 18 columns):
 #   Column                                                  Non-Null Count  Dtype  
---  ------                                                  --------------  -----  
 0   Timestamp                                               1253 non-null   object 
 1   Age                                                     1226 non-null   float64
 2   Gender                                                  1243 non-null   object 
 3   City                                                    1253 non-null   object 
 4   Position                                                1247 non-null   object 
 5   Total years of experience                               1237 non-null   object 
 6   Years of experience in Germany                          1221 non-null   object 
 7   Seniority level                                         1241 non-null   object 
 8   Your main technology / programming lan

In [190]:
df.head()

,Timestamp,Age,Gender,City,Position,Total years of experience,Years of experience in Germany,Seniority level,Your main technology / programming language,Other technologies/programming languages you use often,Yearly brutto salary (without bonus and stocks) in EUR,Yearly bonus + stocks in EUR,Number of vacation days,Employment status,Сontract duration,Main language at work,Company size,Company type
0,24/11/2020 11:14:15,26.0,Male,Munich,Software Engineer,5,3,Senior,TypeScript,"Kotlin, Javascript / Typescript",80000.0,5000,30,Full-time employee,Unlimited contract,English,51-100,Product
1,24/11/2020 11:14:16,26.0,Male,Berlin,Backend Developer,7,4,Senior,Ruby,NaN,80000.0,NaN,28,Full-time employee,Unlimited contract,English,101-1000,Product
2,24/11/2020 11:14:21,29.0,Male,Berlin,Software Engineer,12,6,Lead,Javascript / Typescript,"Javascript / Typescript, Docker",120000.0,120000,30,Self-employed (freelancer),Temporary contract,English,101-1000,Product
3,24/11/2020 11:15:24,28.0,Male,Berlin,Frontend Developer,4,1,Junior,Javascript,NaN,54000.0,NaN,24,Full-time employee,Unlimited contract,English,51-100,Startup
4,24/11/2020 11:15:46,37.0,Male,Berlin,Backend Developer,17,6,Senior,C# .NET,".NET, SQL, AWS, Docker",62000.0,NaN,29,Full-time employee,Unlimited contract,English,101-1000,Product


### Removing Low Frequency Data Points In Categorical Features

In [191]:
def replace_low_freq(d, col, threshold=10, replacement='other'):
    value_counts = d[col].value_counts() 
    to_remove = value_counts[value_counts <= threshold].index
    tmp = d[col].replace(to_replace=to_remove, value=replacement)
    return tmp
df["Your main technology / programming language"] = replace_low_freq(df, "Your main technology / programming language", 10, "Other")
df["Position "] = replace_low_freq(df, "Position ", 10, "Other")
df["Other technologies/programming languages you use often"] = replace_low_freq(df, "Other technologies/programming languages you use often", 10, "Other")
df["City"] = replace_low_freq(df, "City", 10, "Other")
df["Employment status"] = replace_low_freq(df, "Employment status", 10, "Other")
df["Сontract duration"] = replace_low_freq(df, "Сontract duration", 10, "Other")
df["Main language at work"] = replace_low_freq(df, "Main language at work", 10, "Other")
df["Your main technology / programming language"].replace({"Javascript": "JavaScript", "Python ":"Python"}, inplace=True)


### Cleaning Values For Numerical Features

In [192]:
#Convert non-numerical values into NaN values for numerical features
df_EDA = df.drop(columns={'Timestamp'})
df_EDA['Age'] = pd.to_numeric(df['Age'], errors="coerce")
df_EDA['Age'].fillna(df_EDA['Age'].mean(), inplace=True)

df_EDA['Total years of experience'] = pd.to_numeric(df['Total years of experience'], errors="coerce")
df_EDA['Total years of experience'].fillna(df_EDA['Total years of experience'].mean(), inplace=True)

df_EDA['Years of experience in Germany'] = pd.to_numeric(df['Years of experience in Germany'], errors="coerce")
df_EDA['Years of experience in Germany'].fillna(df_EDA['Years of experience in Germany'].mean(), inplace=True)

df_EDA['Yearly brutto salary (without bonus and stocks) in EUR'] = pd.to_numeric(df['Yearly brutto salary (without bonus and stocks) in EUR'], errors="coerce")
df_EDA['Yearly brutto salary (without bonus and stocks) in EUR'].fillna(df_EDA['Yearly brutto salary (without bonus and stocks) in EUR'].median(), inplace=True) #Median is a better metric for salary due to logarithmic distribution.

df_EDA['Yearly bonus + stocks in EUR'] = pd.to_numeric(df['Yearly bonus + stocks in EUR'], errors="coerce")
df_EDA['Yearly bonus + stocks in EUR'].fillna(df_EDA['Yearly bonus + stocks in EUR'].median(), inplace=True) #Median is a better metric for salary due to logarithmic distribution.

df_EDA['Number of vacation days'] = pd.to_numeric(df['Number of vacation days'], errors="coerce")
df_EDA['Number of vacation days'].fillna(df_EDA['Number of vacation days'].mean(), inplace=True)

#Create a column for Total Compensation
df_EDA['Total Compensation'] = df_EDA['Yearly brutto salary (without bonus and stocks) in EUR'] + df_EDA['Yearly bonus + stocks in EUR']
df_EDA = df_EDA.drop(columns={'Yearly brutto salary (without bonus and stocks) in EUR', 'Yearly bonus + stocks in EUR'}) #Remove these features to remove collinearity with total compensation



In [193]:
#For categorical features, convert null values into 'N/A' string for one-hot encoding purposes. 
df_EDA.fillna('N/A',inplace=True)

In [194]:
#Remove outlier
df_EDA = df_EDA[df_EDA['Total Compensation']<=1000000]

In [195]:
#get dummies, create x and y variable, split and train
df_EDA1 = pd.get_dummies(df_EDA, drop_first=True)
y = np.array(df_EDA1["Total Compensation"]).reshape(-1,1)
x = np.array(df_EDA1.drop(columns=["Total Compensation"]))
xtrain,xtest,ytrain,ytest = train_test_split(x, y)



In [196]:
#Generate initial linear regressionmodel
model1 = LinearRegression().fit(xtrain,ytrain)

#Get RMSE and R-squared
r_sq = model1.score(xtest, ytest)
print('R-squared:', r_sq)

ypred = model1.predict(xtest)
print('RMSE:', mean_squared_error(ypred, ytest, squared=False))


R-squared: -3.3438480047566303e+19
RMSE: 289537992459353.7


### Model Testing

In [197]:
scaler_ = MinMaxScaler()
estimator_ = RandomForestRegressor(n_jobs=-1)
pipe_ = Pipeline(steps=[("scaler", scaler_), ("forest", estimator_)])

params_ = {'forest__max_depth':[5,6,7,8,9], 
            "forest__n_estimators":[75,100,125],
            "forest__max_features":[7,8,9,10,11,12,13,14]}
 
model2 = GridSearchCV(pipe_, param_grid=params_, cv=5, n_jobs=-1) 
model2.fit(xtrain, ytrain.ravel())
best_ = model2.best_estimator_
print(best_.score(xtest, ytest))
print(best_)

0.09347783186796677
Pipeline(steps=[('scaler', MinMaxScaler()),
                ('forest',
                 RandomForestRegressor(max_depth=9, max_features=13,
                                       n_estimators=125, n_jobs=-1))])


In [202]:


numeric_features = ['Age','Total years of experience','Years of experience in Germany',
       'Number of vacation days']
numeric_transformer = Pipeline( steps=[
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", MinMaxScaler())
    ])

categorical_features = ['Gender', 'City', 'Position ',
       'Your main technology / programming language',
       'Other technologies/programming languages you use often','Employment status', 'Сontract duration',
       'Main language at work', 'Company size', 'Company type',
       ]
categorical_transformer = OneHotEncoder(handle_unknown="ignore")

preprocessor = ColumnTransformer( remainder='drop',
        transformers=[
        ("num", numeric_transformer, numeric_features),
        ("cat", categorical_transformer, categorical_features),
    ])


y2 = df_EDA1["Total Compensation"]
x2 = df_EDA1.drop(columns={"Total Compensation"})


l1_model = Lasso()
l2_model = Ridge()
pipeline_steps = [('pre', preprocessor),('model', l2_model) ]
pipe = Pipeline(pipeline_steps)

X_train3, X_test3, y_train3, y_test3 = train_test_split(x, y)


pipe.fit(X_train3, y_train3)
cvs = cross_val_score(pipe, X_train3, y_train3.ravel(), cv=5)
print('Training CrossVal Score:', cvs)
print("Average Train Score:", np.mean(cvs))
print('Testing score:', pipe.score(X_test3, y_test3))

Training CrossVal Score: [ 0.01275409  0.02025032  0.0646277  -2.02777651  0.04008588]
Average Train Score: -0.37801170314904836
Testing score: -0.07434602141436786


In [205]:

x = df_EDA1.drop(columns=["Total Compensation"])
var_th = VarianceThreshold(.2)
print(x.shape)
post_vt = var_th.fit_transform(x)
print(post_vt.shape)

mask = var_th.get_support()
new_features =  x.columns[mask]
print(new_features)

(1250, 155)
(1250, 11)
Index(['Age', 'Total years of experience', 'Years of experience in Germany',
       'Number of vacation days', 'Position _Software Engineer',
       'Seniority level_Middle', 'Seniority level_Senior',
       'Your main technology / programming language_Other',
       'Other technologies/programming languages you use often_Other',
       'Company size_101-1000', 'Company type_Product'],
      dtype='object')


In [206]:
scaler = StandardScaler()
estimator = RandomForestClassifier(n_jobs=-1)
pipe = Pipeline(steps=[("scaler", scaler), ("forrest", estimator)])

params = {'forrest__max_depth':[5,6,7,8,9]}
 
clf = GridSearchCV(pipe, param_grid=params, cv=5, n_jobs=-1) 
clf.fit(xtrain, ytrain.ravel())
best = clf.best_estimator_
print(best.score(xtest, ytest))
print(best)

c:\Users\billg\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:378: FitFailedWarning: 
20 fits failed out of a total of 25.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
20 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\billg\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\billg\anaconda3\lib\site-packages\sklearn\pipeline.py", line 382, in fit
    self._final_estimator.fit(Xt, y, **fit_params_last_step)
  File "c:\Users\billg\anaconda3\lib\site-packages\sklearn\ensemble\_forest.py", line 371, in fit
    y, expanded_class_weight = self._validate_y_class_weight(

ValueError: Unknown label type: 'continuous'

# Answers and Explainations
<li> Initial linear regression without feature selection or Gridsearch resulted in low accuracy (r-squared = -0.003 and RMSE was huge)

### Results
<li> Random Forest Regressor seems to give the best accuracy. 
### Feature Selection Activities
<li> t
### Hyperparameter Changes